In [1]:
import proteinmpnn.run
import proteinmpnn.utils.misc

from proteinmpnn.run import load_protein_mpnn_model, set_seed, nll_score
from proteinmpnn.data import BackboneSample, untokenise_sequence
from proteinmpnn.utils.misc import find_files

import numpy as np

/rds/user/ked48/hpc-work/protein-diffusion/proteinmpnn_wrapper/src/proteinmpnn/constants.py


# Manual mode (more control)

In [2]:
model = load_protein_mpnn_model(model_type="ca")

2023-07-20 17:39:20,063 INFO:
	Number of edges: 48 [in load_protein_mpnn_model at run.py:54]
2023-07-20 17:39:20,063 INFO:
	Training noise level: 0.2A [in load_protein_mpnn_model at run.py:56]


In [3]:
# ... add backbones from PDB files
pdb_files = find_files("./data", ".pdb")  # Or numpy or torch objects
backbones = [BackboneSample.load_any(f, ca_only=True) for f in pdb_files]

# ...add backbones from numpy files / torch files
backbones += [BackboneSample(bb_coords=np.random.rand(10, 3), 
                             ca_only=True)
            ]
                           
# ... specify a sequence motif to be fixed (via res_mask -- 0's are fixed)
backbones += [BackboneSample(bb_coords=np.random.rand(10, 3), 
                             ca_only=True, 
                             res_name="MXXXACXGXX", 
                             res_mask=np.array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1]))
            ]

backbones

2023-07-20 17:39:23,889 INFO:
	Found 2 files with extension .pdb in ./data. [in find_files at misc.py:43]


NameError: name 'np' is not defined

In [ ]:
with torch.inference_mode():
    set_seed(39)

    samples = [model.sample(
        randn=torch.randn(1, backbone.n_residues), 
        **backbone.to_protein_mpnn_input("sampling")) 
        for backbone in backbones
    ]

    scores = []
    for sample, backbone in zip(samples, backbones):
        inpt = backbone.to_protein_mpnn_input("scoring")
        inpt["decoding_order"] = sample["decoding_order"]
        inpt["S"] = sample["S"]
        log_probs = model(randn=torch.randn(1, backbone.n_residues), 
                      use_input_decoding_order=True, 
                      **inpt)
        sample["nll_score"] = _nll_score(sample["S"], log_probs, mask=inpt["mask"])
        sample["prob"] = torch.exp(-sample["nll_score"])


In [ ]:
# NOTE the fixed motif
print(untokenise_sequence(samples[0]["S"]))
print(samples[0]["prob"])

TLTLKQTIANQYIKAFERQRSDQCKKCVHPLTIWTVQGWERKREEMVQFVEDMMAKGISWEFQAYERIGVIYDYDAKRQADGVVSFDLYKIEVIEDVIPIIYGNHK
tensor([0.0949])


In [ ]:

# NOTE the fixed motif
print(untokenise_sequence(samples[-2]["S"]))
print(samples[-2]["prob"])

DRRRKDLYNS
tensor([0.0965])


In [ ]:
# NOTE the fixed motif
print(untokenise_sequence(samples[-1]["S"]))
print(samples[-1]["prob"])
samples[-1]

MAYQACEGAD
tensor([0.0395])


{'S': tensor([[10,  0, 19, 13,  0,  1,  3,  5,  0,  2]]),
 'probs': tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0866, 0.0116, 0.0780, 0.0471, 0.0160, 0.2236, 0.0251, 0.0112,
           0.0391, 0.0266, 0.0115, 0.0414, 0.0781, 0.0329, 0.0384, 0.1258,
           0.0597, 0.0189, 0.0100, 0.0186, 0.0000],
          [0.0960, 0.0166, 0.0746, 0.0486, 0.0469, 0.1240, 0.0383, 0.0261,
           0.0327, 0.0455, 0.0216, 0.0467, 0.0405, 0.0367, 0.0400, 0.0937,
           0.0585, 0.0308, 0.0205, 0.0616, 0.0000],
          [0.0065, 0.0045, 0.0059, 0.0044, 0.0063, 0.0075, 0.0041, 0.0041,
           0.0050, 0.0059, 0.0040, 0.0057, 0.0058, 0.0048, 0.0052, 0.0064,
           0.0058, 0.0045, 0.0046, 0.8991, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

# One-function call

In [ ]:
# TODO (wrap the above in a function)